# Generate offset string in hal setting file, and corresponding shutter file

By Pu Zheng

2021.5.31

In [1]:
# import required packages
import os, sys, time
import numpy as np

# generate the offset string

paste the output string into 
</locked>
  	<hardware_z_scan>
		<z_offsets type="string">

# required parameters
num_steps = 50
step_size = 0.25
num_channels = 3

In [2]:
# required parameters PolyT
num_steps = 40
step_size = 0.5
num_channels = 4

# required parameters for DAPI and 488 only
num_steps = 13 #13
step_size = 1 #1
num_channels = 2 #3

# required parameters RNA
num_steps = 13
step_size = 1
num_channels = 2

In [3]:
offset_string = []
start_z = 0 #- (num_steps-1) * step_size / 2
for _i in np.arange(num_steps):
    offset_string.extend([str(round(start_z+_i*step_size, 2))]*num_channels)

In [4]:
# copy the output of this box into the hal setting:
offset_string = ','.join(offset_string)
print(offset_string)

0.0,0.0,0.0,0.0,0.5,0.5,0.5,0.5,1.0,1.0,1.0,1.0,1.5,1.5,1.5,1.5,2.0,2.0,2.0,2.0,2.5,2.5,2.5,2.5,3.0,3.0,3.0,3.0,3.5,3.5,3.5,3.5,4.0,4.0,4.0,4.0,4.5,4.5,4.5,4.5,5.0,5.0,5.0,5.0,5.5,5.5,5.5,5.5,6.0,6.0,6.0,6.0,6.5,6.5,6.5,6.5,7.0,7.0,7.0,7.0,7.5,7.5,7.5,7.5,8.0,8.0,8.0,8.0,8.5,8.5,8.5,8.5,9.0,9.0,9.0,9.0,9.5,9.5,9.5,9.5,10.0,10.0,10.0,10.0,10.5,10.5,10.5,10.5,11.0,11.0,11.0,11.0,11.5,11.5,11.5,11.5,12.0,12.0,12.0,12.0,12.5,12.5,12.5,12.5,13.0,13.0,13.0,13.0,13.5,13.5,13.5,13.5,14.0,14.0,14.0,14.0,14.5,14.5,14.5,14.5,15.0,15.0,15.0,15.0,15.5,15.5,15.5,15.5,16.0,16.0,16.0,16.0,16.5,16.5,16.5,16.5,17.0,17.0,17.0,17.0,17.5,17.5,17.5,17.5,18.0,18.0,18.0,18.0,18.5,18.5,18.5,18.5,19.0,19.0,19.0,19.0,19.5,19.5,19.5,19.5


# generate the shutter file

In [5]:
all_channels= [750, 647, 561, 488, 405]

#ch_ids = [0, 1, 2, 3, 4]
#ch_ids = [0, 3, ]
#ch_ids = [2, 3, 4]
#ch_ids = [0, 1, 3, 4]
#ch_ids = [0, 1, 3]
#ch_ids = [2, 4]
ch_ids = [0, 1, 2, 3]
#ch_ids = [1, 3]

In [6]:
full_shutter_str = ""

## header
header = f"""<?xml version="1.0" encoding="ISO-8859-1"?>
<repeat>
  <oversampling>1</oversampling>
  <frames>{num_steps*num_channels}</frames>
  """
# append header
full_shutter_str += header
## loop through channels
for _i  in np.arange(num_steps):
    full_shutter_str += """
  """
    for _j, _ch in enumerate(ch_ids[:num_channels]):
        # determine start time and end time
        _start_time = _i * num_channels + _j
        _end_time = _i * num_channels + _j + 1
        # determine _channel string
        if _ch < 0:
            continue
        else:
            _ch_str = f"""<event>
    <channel>{_ch}</channel>
    <power>1</power>
    <on>{_start_time:.1f}</on>
    <off>{_end_time:.1f}</off>
    <color>255,255,255</color>
  </event>
  """
        # append
        full_shutter_str += _ch_str

# append the tail
full_shutter_str += """
</repeat>"""

In [7]:
print(full_shutter_str)

<?xml version="1.0" encoding="ISO-8859-1"?>
<repeat>
  <oversampling>1</oversampling>
  <frames>160</frames>
  
  <event>
    <channel>0</channel>
    <power>1</power>
    <on>0.0</on>
    <off>1.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>1</channel>
    <power>1</power>
    <on>1.0</on>
    <off>2.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>2</channel>
    <power>1</power>
    <on>2.0</on>
    <off>3.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>3</channel>
    <power>1</power>
    <on>3.0</on>
    <off>4.0</off>
    <color>255,255,255</color>
  </event>
  
  <event>
    <channel>0</channel>
    <power>1</power>
    <on>4.0</on>
    <off>5.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>1</channel>
    <power>1</power>
    <on>5.0</on>
    <off>6.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>2</channel>
    <power>1</power>
    <on>6.0</on>
    <off

## save this shutter file

In [8]:
parent_folder = r'C:\Users\neoSTORM6\Documents\STORM6\example_scripts\hal'
save_folder = os.path.join(parent_folder, f'{num_steps}_steps_n{int(step_size*1000)}')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    print(f"creating save folder: {save_folder}")
save_basename = "shutter_" + '_'.join([str(all_channels[_id]) for _id in ch_ids[:num_channels]]) + f'_s{num_steps}.xml'
save_filename = os.path.join(save_folder, save_basename)
overwrite = False
if not os.path.isfile(save_filename) or overwrite:
    print(f"-- save the shutter to file: {save_filename}")
    # write
    with open(save_filename, 'w') as _fp:
        _fp.write(full_shutter_str)
else:
    print(f"-- shutter file: {save_filename} already exists, skip")

-- save the shutter to file: C:\Users\neoSTORM6\Documents\STORM6\example_scripts\hal\40_steps_n500\shutter_750_647_561_488_s40.xml


In [9]:
offset_save_basename = "offsets_" + '_'.join([str(all_channels[_id]) for _id in ch_ids[:num_channels]]) + f'_s{num_steps}.txt'
offset_save_filename = os.path.join(save_folder, offset_save_basename)
overwrite = False
if not os.path.isfile(offset_save_filename) or overwrite:
    print(f"-- save the shutter to file: {offset_save_filename}")
    # write
    with open(offset_save_filename, 'w') as _fp:
        _fp.write(offset_string)
else:
    print(f"-- shutter file: {offset_save_filename} already exists, skip")

-- save the shutter to file: C:\Users\neoSTORM6\Documents\STORM6\example_scripts\hal\40_steps_n500\offsets_750_647_561_488_s40.txt
